In [ ]:
import jupyter_drawing_pad as jd
import ipymaterialui as m
from matplotlib import lines
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from fastai.vision.all import *
from fastai.vision.widgets import *
import random

In [ ]:
learner = load_learner("model.pkl",cpu=True)

In [ ]:
widget = jd.CustomBox()

In [ ]:
def toggle_menu(widget, event, data):
    widget.open_ = not widget.open_

In [ ]:
def split_df(df):
    df_tmp = []
    start=0
    for j in np.where(df['time'].diff()>80)[0]:
        df_tmp.append(df.loc[start:j-1,:])
        start = j
    df_tmp.append(df.loc[start:,:])
    return df_tmp

In [ ]:
def create_image(img_data):
    xmin,xmax,ymin,ymax = np.min(img_data[0]),np.max(img_data[0]),np.min(img_data[1]),np.max(img_data[1])
    df = pd.DataFrame.from_records(zip(*img_data),columns=['x','y','time'])
    df_tmp = split_df(df)
    fig, ax = plt.subplots(figsize=(1,1),dpi=256)
    for stroke in df_tmp:
        x = np.array(stroke.to_records(index=False).tolist())[:,0]
        y = np.array(stroke.to_records(index=False).tolist())[:,1]
        line = lines.Line2D(x,y,lw = 1, color ='k', 
                          axes = ax, alpha = 1)
        ax.add_line(line)
    ax.set_xlim(xmin-10,xmax+10)
    ax.set_ylim(ymin-10,ymax+10)
    ax.axis('off')
    return fig

In [ ]:
def toggle_modal(widget, event, data):
    modal1.open_ = not modal1.open_

In [ ]:
def clear_draw(e, w, d):
    widget.clean()
def toggle_dia(widget, event, data):
    dialogue_save.open_ = not dialogue_save.open_
path = "test.png"
#dialogue_save.on_event('onClose', toggle_dia)+
title_tmp = ['Did you draw?',"Is it a?","Isn't it a?"]
dialogue_title = m.DialogTitle(children="did you draw?")
dialogue_content = m.DialogContent(children=['{}'.format(path)])
dialogue_save = m.Dialog(open_=False,children=[dialogue_title,dialogue_content])
dialogue_save.on_event('onClose',toggle_menu)
def submit(e, w, d):
    img_data = widget.drawing_pad.data
    fig = create_image(img_data)
    path = "test.png"
    fig.savefig(path, bbox_inches='tight')
    plt.close()
    with open(path,'rb') as f:
        doc = f.read()
    img = PILImage.create(doc)
    pred,pred_idx,probs = learner.predict(img)
    dialogue_title= m.DialogTitle(children='{}'.format(random.choice(title_tmp)))
    text = m.DialogContentText(children='{}'.format(str(pred).title()))
    text.style_ = {"font-weight":"bold",'font-size':'200%','font-family':'Georgia'}
    dialogue_content.children = text
    dialogue_save.children = [dialogue_title,dialogue_content]
    dialogue_save.open_ = not dialogue_save.open_
button1 = m.Button(color='primary', variant='contained', children=[m.Icon(style_={'marginRight': '5px'}, children='clear'),'clear'])
button2 = m.Button(color='primary', variant='contained', children=[m.Icon(style_={'marginRight': '5px'}, children='send'),'Submit'])
button1.on_event('onClick', clear_draw)
button2.on_event('onClick',submit)
button2.full_width=True
button1.full_width=True
#button2_g = m.Grid(children=[button2,dialogue_save])
button2_g = m.Grid(children=button2)
button1_g = m.Grid(children=button1)
submit_but = m.List(children=[
    button2, dialogue_save,button1
])

In [ ]:
button2_g.item=True;button1_g.item=True;button2_g.xs=1;button1_g.xs=1
dialogue_save.item=True

In [ ]:
button1_g.item=True
button2_g.item=True
container_1_sub = m.Grid(children=[button2_g,dialogue_save,button1_g])

In [ ]:
container_1_sub.spacing=4;container_1_sub.container=True
container_1_sub.justify='center'

In [ ]:
container_2 = m.Grid(children=widget.drawing_pad)

In [ ]:
container_2.item=True;container_2.xs=16

In [ ]:
container_total = m.Grid(children=[container_2,container_1_sub])

In [ ]:
container_total.spacing=0

In [ ]:
container_total.direction = 'column'
container_total.align_content='center'
container_total.container=True

In [ ]:
container_total

In [ ]:
def toggle_menu(widget, event, data):
    widget.open_ = not widget.open_
def toggle_ahead(widget,event,data):
    dialogue.open_ = not dialogue.open_
go_ahead = m.DialogActions(children=m.Button(color='primary', children=['Lets Go']))
dialogue_title = m.DialogTitle(children="Pictionary App")
#dialogue_content = m.DialogContent(children=["Ever played pictionary and wondered if your friends are bad at identifying or you're bad at doodling.",
#       "Wait no more go ahead and check if an",m.Html(tag='p',children=['A.I'],style_={"font-weight":"bold"}) ,"can get your doodle",m.Html(tag='div',children=['{}'.format(", ".join(['a','b']))],style_={"font-weight":"bold"})])
dialogue_content = m.DialogContent(children=["Ever played pictionary and wondered if your friends are bad at identifying or you're bad at doodling. \
        Wait no more!!! Go ahead and check if an A.I an get your doodle.Currently supported doodles are {}".format(",".join(learner.dls.vocab)) ])
dialogue =m.Dialog(open_=True,children=[dialogue_title,dialogue_content,go_ahead])
go_ahead.on_event('onClick',toggle_ahead)
dialogue.on_event('onClose',toggle_menu)
dialogue

In [ ]:
sb = m.Snackbar(open_=True, message=m.Html(tag='div',children=['Made with ❤️'],style_={'position': 'relative', 'left': '100%','right': '100%'}, ))
sb.auto_hide_duration=10000
sb.anchor_origin={ "vertical": 'bottom', "horizontal": 'center' }
sb.on_event('onClose',toggle_menu)
sb